In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter

In [2]:
## loading files into proper dataframes
x_train = pd.read_csv('../raw_data/train.csv')
del x_train['ID']
y_train = pd.read_csv('../raw_data/train.csv',usecols=['y'])
x_test = pd.read_csv('../raw_data/test.csv')
del x_test['ID']

In [3]:
## Checking for missing data
missing = x_train.isnull().sum(axis=0).reset_index()
missing.columns = ['col_name','count']
missing = missing[missing['count']>0]
print missing.head()

missing = x_test.isnull().sum(axis=0).reset_index()
missing.columns = ['col_name','count']
missing = missing[missing['count']>0]
print missing.head()

del missing
## NO missing data !

Empty DataFrame
Columns: [col_name, count]
Index: []
Empty DataFrame
Columns: [col_name, count]
Index: []


In [4]:
## Deleting columns which contain only 0s. Reference - https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-mercedes
del_list = ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
for i in del_list:
    del x_train[i]
    del x_test[i]

In [5]:
## Finding unique values of colmuns with data type objects.
unique_dict = {}
for i in x_train.columns:
    if x_train[i].dtype == object:
        uni_val = str(sorted(x_train[i].unique().tolist()))
        #print type(uni_val)
        temp_list = unique_dict.get(uni_val,[])
        temp_list.append(i)
        unique_dict[uni_val] = temp_list
        
for i,j in unique_dict.items():
    print("Unique val and size",i,len(i))
    print("Cols ",j)
    print "----------------------"

('Unique val and size', "['a', 'aa', 'ab', 'ac', 'ad', 'af', 'ai', 'aj', 'ak', 'al', 'am', 'ao', 'ap', 'aq', 'as', 'at', 'au', 'aw', 'ax', 'ay', 'az', 'b', 'ba', 'bc', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']", 257)
('Cols ', ['X0'])
----------------------
('Unique val and size', "['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']", 60)
('Cols ', ['X6'])
----------------------
('Unique val and size', "['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']", 125)
('Cols ', ['X8'])
----------------------
('Unique val and size', "['a', 'aa', 'ab', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']", 137)
('Cols ', ['X1'])
----------------------
('Unique val and size', "['a', 'aa', 'ac', 'ae', 'af', 'ag', 'ah', 'ai', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as',

In [6]:
## changing labels to ints. Labels corresponding to higher mean of y get high values.
cols = x_train.columns
for i in cols:
    label = []
    replace_dict = {}
    if x_train[i].dtype == object:
        uni_labels = x_train[i].unique().tolist()
        for j in uni_labels:
            avg = x_train[x_train[i]==j]['y'].mean()
            label.append([avg,j])
        label = sorted(label, key=itemgetter(0))
        for k in range(0,len(label)):
            replace_dict[label[k][1]] = k
            
        test_labels = x_test[i].unique().tolist()
        for k in test_labels:
            if not k in replace_dict:
                replace_dict[k] = 0
        x_train[i].replace(replace_dict,inplace=True)
        x_test[i].replace(replace_dict,inplace=True)
        
        del label[:]
        del replace_dict
            

In [7]:
del x_train['y']
x_train.to_csv('../processed_data/pre_x_train.csv',index=False)
x_test.to_csv('../processed_data/pre_x_test.csv',index=False)
y_train.to_csv('../processed_data/y_train.csv',index=False)